In [ ]:
!pip install langchain boto3
!pip install transformers
!pip install sentence_transformers

In [ ]:
pip install langchain==0.0.224

In [ ]:
pip install boto3 --upgrade

In [ ]:
!pip install ~/SageMaker/botocore-1.29.162-py3-none-any.whl
!pip install ~/SageMaker/boto3-1.26.162-py3-none-any.whl

In [ ]:
pip install pypdf

In [ ]:
pip install faiss-cpu

In [ ]:
import boto3
from langchain.retrievers import AmazonKendraRetriever
from typing import List
from typing import Dict
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
#from langchain import SagemakerEndpoint, LLMChain
from langchain import LLMChain
from langchain.llms.bedrock import Bedrock
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json

from langchain.docstore.document import Document
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [ ]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
#        return response_json["vectors"]
        return response_json[0]["generated_text"]



content_handler = ContentHandler()


In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

#llm2 = Bedrock(
#model_id="anthropic.claude-v1",
#model_kwargs={'max_tokens_to_sample': 300})

maxTokensToSample = 300
temp = 0.6
topP = 0.9
topK= 5

modelArgs = {'max_tokens_to_sample': int(maxTokensToSample), 'temperature':float(temp), "top_k":int(topK),"top_p": float(topP),"stop_sequences":[]}
#modelArgs = {'maxTokens': int(maxTokensToSample), 'temperature':float(temp), "topP": float(topP),"stop_sequences":[]}
#modelArgs = {'maxTokens': int(maxTokensToSample) , 'temperature':float(temp), "topP": float(topP)}

llm2 = Bedrock(
model_id="anthropic.claude-v1",
model_kwargs=modelArgs)

#llm2.debug = True

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Document2.pdf")
pages = loader.load_and_split()

In [ ]:
pages[0]

In [ ]:
from langchain.vectorstores import FAISS
#from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import BedrockEmbeddings


faiss_index = FAISS.from_documents(pages, BedrockEmbeddings())



In [ ]:
llm_query= "how do we bake cookies ?"

In [ ]:
docs = faiss_index.similarity_search(llm_query, k=5)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])



In [ ]:
prompt_template = """
Human:{context} {question}
Assistant: 
  """
  
PROMPT = PromptTemplate(
      template=prompt_template, input_variables=["context", "question"]
  )
  
chain = load_qa_chain(llm=llm2, prompt=PROMPT, verbose=True)
output = chain({"input_documents":docs, "question": llm_query}, return_only_outputs=False)
print(output['output_text'])
  